In [1]:
import os
import numpy as np
import glob
from tess_sip import SIP
from astropy.coordinates import SkyCoord
from astropy import units as u
import lightkurve as lk
from astroquery.mast import Tesscut
import tess_cpm
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

sys.path.insert(0, "../")
sys.path.insert(0, "../../Packages/eleanor/")
import eleanor

import warnings
warnings.filterwarnings('ignore')

from michael import janet

In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

index = uni[uni.source_id == 5310986619293965440].index[0]

In [3]:
j = janet.boot(uni, index=index, pipeline='tess-sip-detrended')
j.run(period_range = (2, 20))
plt.show()

Target DR3 ID 5310986619293965440 has tesscut data available on MAST for Sectors 8, 9, 10, 35, 36
Data already downloaded for Sector 8.
Data already downloaded for Sector 9.
Data already downloaded for Sector 10.
Data already downloaded for Sector 35.
Data already downloaded for Sector 36.


NameError: name 'sectsr' is not defined

In [ ]:
glob.glob(f'{j.data.path}/tess-s001*')[0]

In [ ]:
sys.exit()glob.

In [ ]:
j = janet.boot(uni, index=index, pipeline='eleanor')
j.run(period_range = (2, 20))
plt.show()